In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
I = cv2.imread('data/cm1.png', cv2.IMREAD_GRAYSCALE) # wczesniejsza
J = cv2.imread('data/cm2.png', cv2.IMREAD_GRAYSCALE) # pozniejsza

# resize to half
# I = cv2.resize(I, (0,0), fx=0.5, fy=0.5)
# J = cv2.resize(J, (0,0), fx=0.5, fy=0.5)

In [3]:
# show with named window
cv2.imshow('I', I)
cv2.imshow('J', J)

absdiff = cv2.absdiff(I, J)
cv2.imshow('absdiff', absdiff)

# wait for any key to close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
W2 = 3
dx = dy = 3

u = v = np.zeros(I.shape, dtype=np.float32)

for j in range(W2, I.shape[0], 1):
    for i in range(W2, I.shape[1], 1):
        if not (j-W2 >= 0 and j+W2+1 < I.shape[0] and i-W2 >= 0 and i+W2+1 < I.shape[1]):
            continue
        I0 = np.float32(I[j-W2:j+W2+1,i-W2:i+W2+1])
        # print(j-W2, j+W2+1, i-W2, i+W2+1)
        # check J(j,i) surrounding
        smallest_dist = np.inf
        best_i = 0
        best_j = 0

        for jj in range(-dx, dx+1):
            for ii in range(-dy, dy+1):
                if not (j+jj-W2 >= 0 and j+jj+W2+1 < J.shape[0] and i+ii-W2 >= 0 and i+ii+W2+1 < J.shape[1]):
                    continue
                J0 = np.float32(J[j+jj-W2:j+jj+W2+1,i+ii-W2:i+ii+W2+1])
                dist = np.sqrt(np.sum(np.square(I0-J0)))
                if dist < smallest_dist:
                    smallest_dist = dist
                    best_i = ii
                    best_j = jj
        v[j,i] = best_i
        u[j,i] = best_j

# visualize flow using HSV
hsv = np.zeros((I.shape[0], I.shape[1], 3), dtype=np.uint8)
hsv[...,2] = 255
mag, ang = cv2.cartToPolar(np.float32(u), np.float32(v))
hsv[...,0] = ang*180/np.pi/2
hsv[...,1] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
cv2.imshow('flow', rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()